In [ ]:
from libs import notebook_helpers
notebook_helpers.set_covid_data_public()

In [ ]:
import pandarallel
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
from datapublic.common_fields import CommonFields
from libs.qa import data_availability
from libs.datasets.latest_values_dataset import LatestValuesDataset
from libs.datasets.timeseries import TimeseriesDataset

pd.options.display.max_rows = 3000
pd.options.display.max_columns = 3000
pandarallel.pandarallel.initialize(progress_bar=True)

In [ ]:
sources_latest = data_availability.load_all_latest_sources()
all_data = [source.data for source in sources_latest.values()]
all_sources_latest = LatestValuesDataset(pd.concat(all_data))

# Data availability for a specific field across data sources

In [ ]:
not_included_columns = ['fips', 'date', 'state', 'county', 'country', 'aggregate_level']
columns = [column for column in all_sources_latest.data.columns if column not in not_included_columns]


@interact
def show_field_data_sources(field=widgets.Select(options=sorted(columns))):
    display(data_availability.build_data_availability_for_field(all_sources_latest, field))


# Data Availability across all fields for a single data source

In [ ]:
select_widget = widgets.Select(options=list({k: v for k, v in sources_latest.items()}.items()))


@interact
def show_provenance_by_source_sources(dataset=select_widget):
    return data_availability.build_data_availability_report(dataset)